## Importações e Funções de Apoio

In [123]:
import pandas as pd
import os
import re
import warnings
import json
import unicodedata
from io import StringIO
import string

warnings.filterwarnings('ignore')
pd.options.display.float_format = '{:,.2f}'.format

In [124]:
def ajustes(x):
    x = str(x).strip()
    return 0 if x == "-" or x == "R$ -" else x

def strip(x):
    # Converter para string e remover espaços iniciais e finais
    x = str(x).strip()

    # Remover acentos
    x = ''.join(
        c for c in unicodedata.normalize('NFKD', x)
        if not unicodedata.combining(c)
    )

    # Remover pontuações
    x = x.translate(str.maketrans('', '', string.punctuation))
    x = str(x).upper()

    return x

In [125]:
with open(r"C:\Users\divtec-dados\DataspellProjects\Notebooks\Ceasa\Dados\JSON\lista_nomes_produtos_ceasa_online.json", "r", encoding="utf-8") as file:
	dicionario_nomes_produtos = json.load(file)
	for item in dicionario_nomes_produtos:
		dicionario_nomes_produtos[item] = dicionario_nomes_produtos[item]

In [126]:
df_pesos = pd.read_csv(r'C:\Users\divtec-dados\DataspellProjects\Notebooks\Ceasa\Dados\pesos_produtos_ceasa_online.csv', sep=";", decimal=",")
df_pesos['PESO'] = df_pesos['PESO'].astype('float')
df_pesos['CODIGO'] = df_pesos['CODIGO'].astype('string')

## Carregando e ajustes dos dados

In [127]:
caminho = r"C:\Users\divtec-dados\DataspellProjects\Notebooks\Ceasa\Dados\Cotações\Histórico de Cotações Ceasa.csv"
dados_brutos = pd.read_csv(caminho, sep=";", decimal=",")
dados_brutos['DATA'] = pd.to_datetime(dados_brutos['DATA'], format='%d/%m/%Y', dayfirst=True)
df = dados_brutos.copy(deep=True)
df[['MÁXIMO','MÍNIMO','M COMUM']] = df[['MÁXIMO','MÍNIMO','M COMUM']].astype(float)
df['PRODUTO'] = df['PRODUTO'].map(dicionario_nomes_produtos)
df

,DATA,PRODUTO,UNIDADE,MÁXIMO,M COMUM,MÍNIMO
0,2020-01-02,Abacate Comum,KG,5.00,5.00,4.00
1,2020-01-02,Abacaxi Caiena (Hawai),UND,4.00,3.80,3.00
2,2020-01-02,Abacaxi Perola,UND,3.50,3.50,2.50
3,2020-01-02,Abacaxi Terra de Areia,UND,3.00,2.50,2.00
4,2020-01-02,Abobora,KG,1.75,1.50,1.20
...,...,...,...,...,...,...
126164,2024-12-30,Verdes Ruscus,MOL,30.00,30.00,30.00
126165,2024-12-30,Violeta,UND,6.90,6.90,5.00
126166,2024-12-30,Violeta Mini,CX,NaN,NaN,NaN
126167,2024-12-30,Yacon,KG,19.00,15.00,10.56


## Manipulação dos Dados

## Verificação das Datas

## Padronização dos Dados para exportar ao COL

In [128]:
df_export = pd.merge(df, df_pesos, how='left', on=['PRODUTO','UNIDADE'])

In [129]:
df_export.loc[df_export.isna().any(axis=1)]

,DATA,PRODUTO,UNIDADE,MÁXIMO,M COMUM,MÍNIMO,CODIGO,PESO
19442,2020-11-24,Violeta,KG,0.00,0.00,0.00,<NA>,NaN
19601,2020-11-26,Violeta,KG,0.00,0.00,0.00,<NA>,NaN
20356,2020-12-15,Ovo Branco,KG,0.00,0.00,0.00,<NA>,NaN
20384,2020-12-15,Verdes (Samambaia Preta),KG,0.00,0.00,0.00,<NA>,NaN
20545,2020-12-17,Verdes (Samambaia Preta),KG,0.00,0.00,0.00,<NA>,NaN
...,...,...,...,...,...,...,...,...
126144,2024-12-30,Rosa Colorida,MOL,NaN,NaN,NaN,192.0,0.60
126157,2024-12-30,Tomate Rasteiro,KG,NaN,NaN,NaN,204.0,1.00
126158,2024-12-30,Tulipa,UND,NaN,NaN,NaN,206.0,0.30
126162,2024-12-30,Verdes Aspargo Importado(a),MOL,NaN,NaN,NaN,315.0,0.30
